### <font color='red'>变量基本信息</font>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
def get_base_info(train):
    """
    返回 DataFrame
    Args:
        train (_type_): DataFrame
    """
    base_info = []
    for col in train.columns:
        base_info.append((col,  
                        train[col].nunique(), 
                        train[col].isnull().sum() * 100 / train.shape[0], 
                        train[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                        train[col].dtype))
    base_info_df = pd.DataFrame(
        base_info, columns=['Feature', 'unique值个数', '缺失率', '最大同值率', 'type'])
    base_info_df.sort_values('缺失率', ascending=False)
    return base_info_df

### <font color='red'>缺失值分布图</font>

In [ ]:
def plot_missing(train):
    """
    缺失值分布
    """
    missing = train.isnull().sum()
    missing = missing[missing > 0]
    if missing.shape[0] > 0:
        missing.sort_values(ascending=False, inplace=True)
        missing.plot.bar()
    else:
        return 

### <font color='red'>单变量分布（连续变量）</font>

#### 箱形图

In [ ]:
def plot_box(train_data):
    """
    箱形图:主要借助中位数和四分位数来进行计算，以上四分位数+1.5倍四分位距为上界、下四分位数-1.5倍四分位距为下界，超出界限则认为是异常值
    """
    fig = plt.figure(figsize=(60, int(np.ceil(train_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        sns.boxplot(x=train_data[col], width=0.5, ax=ax)
        ax.set_xlabel(col, fontsize=36)
    plt.show()

#### 异常值

孤立森林

In [4]:
def find_outliers_by_isoforest(data, if_plot=True):
    from sklearn.ensemble import IsolationForest
    
    clf = IsolationForest(contamination=0.01, bootstrap=True, random_state=5)
    outlier_index = clf.fit_predict(data)
    outliers = data[outlier_index == -1]
    if if_plot and data.shape[1] >= 2:
        col1 = data.columns[0]
        col2 = data.columns[1]
        sns.scatterplot(data[col1], data[col2], hue=outlier_index, palette='Set1')
    return outliers

距均值3倍标准差

In [ ]:
def find_outliers_by_modelpred(model, X, y, sigma=3):
    """
    样本维度：以样本标签均值-3倍标注差为下界，均值+3倍标准差为上界，来检测是否有超过边界的点
    """
    import pandas as pd
    import matplotlib.pyplot as plt

    def plot_outliers(y, y_pred, z, outliers):
        plt.figure(figsize=(15, 5))
        ax_131 = plt.subplot(1, 3, 1)
        plt.plot(y, y_pred, '.')
        plt.plot(y.loc[outliers], y_pred.loc[outliers], 'ro')
        plt.legend(['Accepted', 'Outliers'])
        plt.xlabel('y')
        plt.ylabel('y_pred')
        
        ax_132 = plt.subplot(1, 3, 2)
        plt.plot(y, y_pred, '.')
        plt.plot(y.loc[outliers], y.loc[outliers] - y_pred.loc[outliers], 'ro')
        plt.legend(['Accepted', 'Outliers'])
        plt.xlabel('y')
        plt.ylabel('y - y_pred')

        ax_133 = plt.subplot(1, 3, 3)
        z.plot.hist(bins=50, ax=ax_133)
        z.loc[outliers].plot.hist(color='r', bins=50, ax=ax_133)
        plt.legend(['Accepted', 'Outliers'])
        plt.xlabel('z')
        plt.ylabel('Frequency')

    model.fit(X, y)
    y_pred = pd.Series(model.predict(X), index=y.index)
    residuals = y - y_pred
    mean_resid = residuals.mean()
    std_resid = residuals.std()
    # cal z statistic, define outliers to be where |z| > sigma
    z = (residuals - mean_resid) / std_resid
    outliers = z[abs(z) > sigma].index
    plot_outliers(y, y_pred, z, outliers)
    return outliers

#### 核密度估计图
训练集、测试集分布是否一致

In [ ]:
def plot_kde(train_data, test_data):
    """
    核密度估计图,是在概率论中用来估计未知的密度函数，属于非参数检验的方法之一。
    通过核密度估计图可以比较直观地看出数据样本本身的分布特征
    """
    fig = plt.figure(figsize=(60, int(np.ceil(test_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in test_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        sns.kdeplot(train_data[col], color='red', shade=True)
        sns.kdeplot(test_data[col], color='blue', shade=True)
        ax.set_xlabel(col, fontsize=36)
        ax.set_ylabel('Frequency')
        ax = ax.legend(['train', 'test'])
    plt.show()

#### 单变量psi值
训练集、测试集分布是否一致  
非监督算法（等频、等距）

In [ ]:
# 计算psi（传入数据可为变量、分数）
def cal_psi(actual, expected, cut_method='equalfrequency', cut_points=[], bins=10):
    """
    args:
        actual：实际数据（线上），Series
        expected：期望数据（线下），Series
        cut_method：分箱方法，'auto'：手动分箱，'equidistance'：等距分箱，'equalfrequency'：等频分箱
        cut_points：切点列表
        bins：分箱数
    return:
        psi_info：各区间的分布及psi
        psi：psi值
    """
    # 1.确定分箱方法
    if cut_method == 'auto':  # 手动分箱
        cut_points = [-np.inf] + cut_points + [np.inf]
    elif cut_method == 'equidistance':  # 等距分箱
        bins = min(bins, expected.nunique())  # 去掉空值后值的个数与给定箱数取小
        cut_points = list(np.linspace(expected.min(), expected.max(), num=bins))
        cut_points = [-np.inf] + cut_points + [np.inf]
    elif cut_method == 'equalfrequency':  # 等频分箱
        bins = min(bins, expected.nunique())  # 去掉空值后值的个数与给定箱数取小
        cut_points = list(np.nanquantile(expected, [i / bins for i in range(1, bins)])) if bins > 1 \
            else list(np.nanquantile(expected, [i / bins for i in range(1, bins + 1)]))
        cut_points = sorted(list(set(cut_points)))
        cut_points = [-np.inf] + cut_points + [np.inf]

    # 2.计算各区间的样本量
    Range = ['missing']
    expected_bins_cnt = [expected.isnull().sum()]
    actual_bins_cnt = [actual.isnull().sum()]
    for i in range(len(cut_points) - 1):
        # print(i)
        # 区间
        left = round(cut_points[i], 2)
        right = round(cut_points[i + 1], 2)
        Range.append('(' + str(left) + ',' + str(right) + ']')
        # 计算expected样本量
        ebins_cnt = ((expected > left) & (expected <= right)).sum()
        expected_bins_cnt.append(ebins_cnt)
        # 计算actual样本量
        abins_cnt = ((actual > left) & (actual <= right)).sum()
        actual_bins_cnt.append(abins_cnt)
    psi_info = pd.DataFrame({'Range': Range, 'Expected_cnt': expected_bins_cnt, 'Actual_cnt': actual_bins_cnt})

    # 3.计算psi
    psi_info['Expected_pct'] = psi_info['Expected_cnt'] / len(expected)
    psi_info['Expected_pct'] = np.where(psi_info['Expected_pct'] == 0, 0.00001, psi_info['Expected_pct'])

    psi_info['Actual_pct'] = psi_info['Actual_cnt'] / len(actual)
    psi_info['Actual_pct'] = np.where(psi_info['Actual_pct'] == 0, 0.00001, psi_info['Actual_pct'])

    psi_info['A-E'] = psi_info['Actual_pct'] - psi_info['Expected_pct']
    psi_info['A/E'] = psi_info['Actual_pct'] / psi_info['Expected_pct']
    psi_info['ln(A/E)'] = np.log(psi_info['A/E'])

    psi_info['PSI'] = psi_info['A-E'] * psi_info['ln(A/E)']
    psi = psi_info['PSI'].sum()

    return psi, psi_info

In [ ]:
def _psi(base_series, test_series, bins=10, min_sample=10):
    base_list = base_series.values
    test_list = test_series.values
    try:
        base_df = pd.DataFrame(base_list, columns=['score'])
        test_df = pd.DataFrame(test_list, columns=['score']) 
        
        # 1.去除缺失值后，统计两个分布的样本量
        base_notnull_cnt = len(list(base_df['score'].dropna()))
        test_notnull_cnt = len(list(test_df['score'].dropna()))
        
        # 空分箱
        base_null_cnt = len(base_df) - base_notnull_cnt
        test_null_cnt = len(test_df) - test_notnull_cnt
        
        # 2.最小分箱数
        q_list = []
        if pd.api.types.is_numeric_dtype(base_series):  # 连续型，按分位数分箱汇总
            if type(bins) == int:
                bin_num = min(bins, int(base_notnull_cnt / min_sample))
                q_list = [x / bin_num for x in range(1, bin_num)]
                break_list = []
                for q in q_list:
                    bk = base_df['score'].quantile(q)
                    break_list.append(bk)
                break_list = sorted(list(set(break_list))) # 去重复后排序
                score_bin_list = [-np.inf] + break_list + [np.inf]
            else:
                score_bin_list = bins
        elif pd.api.types.is_object_dtype(base_series):
                score_bin_list = sorted(list(set(list(base_list) + list(test_list))))  # 离散型，直接按值汇总
        
        # 4.统计各分箱内的样本量
        base_cnt_list = [base_null_cnt]
        test_cnt_list = [test_null_cnt]
        bucket_list = ["MISSING"]
        for i in range(len(score_bin_list)-1):
            left  = round(score_bin_list[i+0], 4)
            right = round(score_bin_list[i+1], 4)
            bucket_list.append("(" + str(left) + ',' + str(right) + ']')
            
            base_cnt = base_df[(base_df.score > left) & (base_df.score <= right)].shape[0]
            base_cnt_list.append(base_cnt)
            
            test_cnt = test_df[(test_df.score > left) & (test_df.score <= right)].shape[0]
            test_cnt_list.append(test_cnt)
        
        # 5.汇总统计结果    
        stat_df = pd.DataFrame({"variable": base_series.name, "bucket": bucket_list, "base_cnt": base_cnt_list, "test_cnt": test_cnt_list})
        stat_df['base_dist'] = stat_df['base_cnt'] / len(base_df)
        stat_df['test_dist'] = stat_df['test_cnt'] / len(test_df)
        
        def sub_psi(row):
            # 6.计算PSI
            base_list = row['base_dist']
            test_dist = row['test_dist']
            # 处理某分箱内样本量为0的情况
            if base_list == 0 and test_dist == 0:
                return 0
            elif base_list == 0 and test_dist > 0:
                base_list = 1 / base_notnull_cnt
            elif base_list > 0 and test_dist == 0:
                test_dist = 1 / test_notnull_cnt
                
            return (test_dist - base_list) * np.log(test_dist / base_list)
        
        stat_df['psi'] = stat_df.apply(lambda row: sub_psi(row), axis=1)
        stat_df = stat_df[['variable', 'bucket', 'base_cnt', 'base_dist', 'test_cnt', 'test_dist', 'psi']]
        
        psi = stat_df['psi'].sum()
        
    except:
        print('error!!!')
        psi = np.nan 
        stat_df = None    
    return psi, stat_df

def unpack_tuple(x):
        if len(x) == 1:
            return x[0]
        else:
            return x

def calculate_psi(base, test, return_frame=True):
    psi = list()
    frame = list()

    if isinstance(test, pd.DataFrame):
        for col in test:
            p, f = _psi(base[col], test[col])
            psi.append(p)
            frame.append(f)

        psi = pd.Series(psi, index = test.columns)
    else:
        psi, frame = _psi(base, test)
    
    res = (psi,)
    if return_frame:
        res += (frame,)
    return unpack_tuple(res)

#### KS检验

In [ ]:
from scipy.stats import ks_2samp

def ks_test(train, test, col, thresh=0.01):
    # < 0.01 # 原假设：独立同分布，p越小越拒绝
    p_val = ks_2samp(train[col], test[col]).pvalue
    if p_val < thresh:
        return False
    else:
        return True

#### 直方图
变量是否服从正态分布

In [ ]:
def plot_hist(train_data):
    """
    直方图,通过表示沿数据范围形成封箱，然后绘制条形以显示落入每个分箱贯彻的次数的数据分布
    结合QQ图,可观察变量是否需要做变换,以更接近正态分布
    Args:
        train_data (_type_): _description_
    """
    import warnings

    warnings.filterwarnings('ignore')
    
    fig = plt.figure(figsize=(60, int(np.ceil(train_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        # sns.histplot(train_data[col], bins=10, kde=True, ax=ax)
        sns.distplot(train_data[col], bins=10, hist=True, kde=True, fit=stats.norm)
        ax.set_xlabel(col, fontsize=36)
    plt.show()

#### QQ图
变量是否服从正态分布

In [ ]:
def plot_qq(train_data):
    """
    绘制QQ图,查看变量是否服从正态分布
    """
    fig = plt.figure(figsize=(60, int(np.ceil(train_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        stats.probplot(train_data[col], plot=plt)
        ax.set_xlabel(col, fontsize=36)
    plt.show()

#### 直方图 + QQ图
变量是否服从正态分布

In [ ]:
def plot_hist_qq(train_data):
    """
    画直方图 + QQ图
    观察变量是否服从正态分布
    Args:
        train_data (_type_): 训练集
    """

    import warnings

    warnings.filterwarnings('ignore')
    
    fig = plt.figure(figsize=(60, int(np.ceil(train_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        # sns.histplot(train_data[col], bins=10, kde=True, ax=ax)
        sns.distplot(train_data[col], bins=10, hist=True, kde=True, fit=stats.norm)
        ax.set_xlabel(col, fontsize=36)

        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        stats.probplot(train_data[col], plot=plt)
        ax.set_xlabel(col, fontsize=36)
    plt.show()

#### Box-Cox变换

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def box_cox_transform(train, test):
    """
    boxcox变换中x必须为一维的正的数组
    y = (x**lmbda - 1) / lmbda,  for lmbda != 0
        log(x),                  for lmbda = 0
    
    Args:
        train (_type_): _description_
        test (_type_): _description_
    """
    scaler = MinMaxScaler()
    all_scaled = scaler.fit(pd.concat([train[test.columns], test], axis=0))
    train_scaled = scaler.transform(train[test.columns])
    test_scaled = scaler.transform(test)
    train_scaled = pd.DataFrame(train_scaled, columns=test.columns)
    test_scaled = pd.DataFrame(test_scaled, columns=test.columns)
    
    for col in train_scaled.columns:
        trans_train_var, lambda_var = stats.boxcox(train_scaled[col].dropna() + 1)
        trans_test_var = stats.boxcox(test_scaled[col].dropna() + 1, lmbda=lambda_var)
        train[col] = trans_train_var
        test[col] = trans_test_var
    return train, test

#### 线性关系图

In [ ]:
def plot_reg(train_data, y_label='target'):
    """
    线性关系图
    """
    fig = plt.figure(figsize=(60, int(np.ceil(train_data.columns.shape[0] / 6) * 50)), dpi=75)  # 4:3
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        sns.regplot(x=col, y=y_label, data=train_data, ax=ax, scatter_kws={'marker': '.', 's': 3, 'alpha': 0.3}, line_kws={'color': 'k'})
        ax.set_xlabel(col, fontsize=36)
        ax.set_ylabel('target')

### <font color='red'>单变量分布（离散变量）</font>

#### 柱状图

In [ ]:
def plot_histogram(train_data):
    """
    柱状图
    """
    i = 0
    for col in train_data.columns:
        i += 1
        ax = plt.subplot(len(train_data.columns), 6, i)
        sns.countplot(train_data[col], data=train_data, ax=ax)
        ax.set_xlabel(col)


### <font color='red'>双变量分布（连续 & 连续）</font>

#### 相关性系数

In [ ]:
def get_corr(data_train, y_label='target', n_corr=10):
    """
    1.plot_kde,剔除分布不一致的变量
    2.计算剩余X与y的相关系数
    Args:
        data_train (_type_): 训练集
        y_label (str, optional): y标签. Defaults to 'target'.
        n_corr (int, optional): 挑选最相关的`n`个变量. Defaults to 10.
    """
    train_corr = data_train.corr()
    cols_k_largest = train_corr.nlargest(n_corr, y_label)[y_label].index
    return cols_k_largest

#### 成对散点图

In [ ]:
def plot_pair_scatter(data):
    """
    成对散点图
    Args:
        data (_type_): _description_
    """
    sns.pairplot(data, kind="reg", diag_kind="kde")

#### 热力图

In [ ]:
def plot_heatmap(train_corr):
    """
    热力图
    Args:
        train_corr (_type_): 相关系数矩阵
    """
    sns.heatmap(train_corr, vmax=.8, square=True, annot=True)

### <font color='red'>双变量分布（离散 & 离散）</font>

#### 柱状图

##### 堆叠柱状图

In [ ]:
def plot_stacked_histogram(data, x_label, y_label1, y_label2):
    """
    堆叠柱状图
    Args:
        data (_type_): DataFrame
        x_label (_type_): 变量分箱
        y_label1 (_type_): 组1标签
        y_label2 (_type_): 组2标签
    """
    s1 = sns.barplot(x=x_label, y=y_label1, data=data, color='red')
    s2 = sns.barplot(x=x_label, y=y_label2, data=data, color='blue')

##### 分组柱状图

In [ ]:
def plot_group_histogram(data, x_label, y_label, group):
    """
    分组柱状图
    Args:
        data (_type_): _description_
        x_label (_type_): 变量分箱
        y_label (_type_): 标签
        group (_type_): 分组依据
    """
    sns.barplot(x =x_label, y = y_label, data = data, hue = group)

### <font color='red'>特征处理</font>

#### 标准化

In [ ]:
from sklearn.preprocessing import StandardScaler

'''
每个样本点都对标准化过程产生影响,适用于有异常值的场景
缩放到均值为0,方差为1
'''

scaler = StandardScaler()

#### 归一化

按行

In [ ]:
from sklearn.preprocessing import Normalizer

'''
对样本进行归一化,映射到[0,1]
'''
scaler = Normalizer(norm='l2')  #! 按行

按列

In [ ]:
from sklearn.preprocessing import MinMaxScaler

'''
映射到[0,1]
'''
scaler = MinMaxScaler()

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

'''
映射到[-1,1]
'''
scaler = MaxAbsScaler()  # x_new = x / max(abs(x))

#### 二值化

In [ ]:
from sklearn.preprocessing import Binarizer

'''
> 阈值映射为1
<= 则映射为0
'''
scaler = Binarizer(threshold=0.0)

#### 哑变量

有序编码

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()  #! 处理y标签专用

One-Hot编码

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

#### 缺失值填充

简单填充

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # mean、medain、most_frequent、constant


模型预测填充  
在每个步骤中,选择一个特征作为输出y,其他所有特征作为输入的X。然后在X和y上训练一个回归器,用来预测y的缺失值

In [1]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor

imputer = IterativeImputer(RandomForestRegressor(), random_state=5)

#### 数据转换

多项式转换

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=False)

函数转换

In [ ]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)

#### 特征选择

<img src="https://img-blog.csdnimg.cn/img_convert/91be57ac7102e168ef93592d1bc45fb8.png" alt="image-20210815163752767" style="zoom:50%;" />

##### 过滤法

##### 应用场景
- 对于回归：
1. f_regression 回归任务的标签/功能之间的F值
2. mutual_info_regression 共同目标的共同信息

- 对于分类：
3. chi2 分类任务的非负特征的卡方统计
4. f_classif 标签/功能之间的ANOVA F值用于分类任务
5. mutual_info_classif 离散目标的相互信息

- 其他方法：
6. SelectPercentile 根据最高分数的百分位数选择功能。
7. SelectFpr 根据误报率测试选择功能。
8. SelectFdr 根据估计的错误发现率选择功能。
9. SelectFwe 根据家庭错误率选择功能。
10. GenericUnivariateSelect 具有可配置模式的单变量特征选择器。

过滤法-相关系数法

In [8]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, r_regression
X, y = load_digits(return_X_y=True, as_frame=True)

select_corr = SelectKBest(r_regression, k=20)
X_new = select_corr.fit_transform(X, y)
X_new

array([[ 9.,  1.,  0., ...,  9.,  7.,  0.],
       [13.,  5.,  0., ...,  3.,  0.,  0.],
       [15., 12.,  0., ...,  1.,  0.,  0.],
       ...,
       [15.,  1.,  0., ...,  4.,  0.,  0.],
       [ 7.,  0.,  0., ..., 16.,  2.,  0.],
       [ 8.,  1.,  0., ..., 12.,  6.,  0.]])

In [12]:
select_corr.get_feature_names_out()

array(['pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_2',
       'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_2', 'pixel_2_5',
       'pixel_2_6', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5',
       'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_5_6', 'pixel_5_7'],
      dtype=object)

In [9]:
X.columns[select_corr.get_support()]

Index(['pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_2',
       'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_2', 'pixel_2_5',
       'pixel_2_6', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5',
       'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_5_6', 'pixel_5_7'],
      dtype='object')

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from scipy.stats import pearsonr
from sklearn.feature_selection import r_regression, f_regression

# pearsonr返回r，相关性系数、p-value（双尾），p值可粗略表示两者不相关的概率，p越小越相关
# score_func = lambda X, y: np.array(list(map(lambda x: pearsonr(x, y), X.T))).T[0]  # r值
def feature_selection_corr(X_train_temp, X_test_temp, y_train, k):
    """
    相关系数：衡量连续变量的同步变化趋势，很多分类问题采用相关系数进行衡量并不能得到一个很好的结果
    在机器学习建模流程中，借助关联度指标进行特征筛选也往往是初筛，并不需要精准的检验结果作为依据，
    例如我们并不会以p值作为特征是否可用的依据（并不只使用那些显著相关的变量），
    而是“简单粗暴”的划定一个范围（例如挑选相关性前100的特征）"""
    # 1.r_regression只会根据输入函数的评分按照由高到低进行筛选，因此输出的特征只是相关系数最大并不是相关系数绝对值最大的特征，
    # 2.r_regression返回结果的特征并未排序
    # KB= SelectKBest(r_regression, k=k)
    # 3.f_regression计算F-Score，基于F-Score的排序结果和基于相关系数绝对值的排序结果一致
    KB= SelectKBest(f_regression, k=k)
    KB.fit_transform(X_train_temp, y_train)
    selected_var = X_train_temp.columns[KB.get_support()]
    # 4.SelectPercentile可以按照比例进行筛选
    # SP = SelectPercentile(f_regression, percentile=30)
    X_train_temp = X_train_temp[selected_var]
    X_test_temp = X_test_temp[selected_var]
    return X_train_temp, X_test_temp

过滤法-方差选择

In [ ]:
def feature_selection_var(X, threshold=0):
    from sklearn.feature_selection import VarianceThreshold

    select_var = VarianceThreshold(threshold=threshold)
    select_var.fit(X)
    selected_var = X.columns[select_var.get_support()]
    return selected_var

过滤法-卡方

In [ ]:
'''离散变量'''
def feature_selection_chi(X, y, threshold=0.05):
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2

    chi_X = X.copy()
    chi_X = chi_X.where(chi_X >= 0, 0)  # 卡方检验不能非负

    chival, pval = chi2(chi_X, y)
    k_chi = pval.shape[0] - (pval > threshold).sum()  # 原假设：两者独立无关，p越小越拒绝
    select_chi = SelectKBest(chi2, k=k_chi)
    select_chi.fit(chi_X, y)
    selected_chi = X.columns[select_chi.get_support()]
    return selected_chi

过滤法-ANOVA方差分析

In [ ]:
'''连续变量'''
def feature_selection_f(X, y, threshold=0.05):
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_classif

    fval, pval = f_classif(X, y)
    k_f = pval.shape[0] - (pval > threshold).sum()
    select_f = SelectKBest(f_classif, k=k_f)
    select_f.fit(X, y)
    selected_f = X.columns[select_f.get_support()]
    return selected_f

过滤法-互信息  
互信息，可以看成是一个随机变量中包含的关于另一个随机变量的信息量

In [ ]:
def feature_selection_mic(X, y, threshold=0):
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import mutual_info_classif as mic

    val = mic(X, y)  # 互信息估计值
    k_mic = val.shape[0] - (val <= threshold).sum()
    select_mic = SelectKBest(mic, k=k_mic)
    select_mic.fit(X, y)
    selected_mic = X.columns[select_mic.get_support()]
    return selected_mic

##### 包装法

包装法-RFE

In [ ]:
'''很慢'''
# def feature_selection_wrapper(clf, X, y, scoring='roc_auc', threshold=[], step=1):
#     import numpy as np
#     import matplotlib.pyplot as plt
#     from sklearn.feature_selection import RFE
#     from sklearn.model_selection import cross_val_score

#     if len(threshold) == 0:
#         threshold = np.linspace(1, int(np.ceil(X.shape[1] / 10)) * 9 + 1, 10)
#         step = np.ceil(X.shape[1] / 10)
#     scores = []
#     for th in threshold:
#         X_wrapper = RFE(clf, n_features_to_select=int(th),
#                         step=step).fit_transform(X, y)
#         score = cross_val_score(clf, X_wrapper, y, scoring=scoring, cv=5, n_jobs=-1, error_score='raise').mean()  # cross_val_score也是测试集分数
#         scores.append(score)
#     plt.plot(threshold, scores)
#     plt.show()
#     return scores

In [ ]:
def feature_selection_wrapper(model, X_train, y_train, X_test, y_test, X_oot, y_oot, scoring='roc_auc'):

    from sklearn.feature_selection import RFE
    from sklearn.model_selection import cross_val_score

    rfe = RFE(model, n_features_to_select=1, step=1, verbose=0)
    rfe.fit(X_train, y_train)
    dic = {k : v for k, v in zip(rfe.ranking_, X_train.columns)}
    feature_ranking = pd.Series(dic).reset_index().rename({'index': 'ranking', 0: 'feature'}, axis=1)
    feature_ranking = feature_ranking.sort_values('ranking', axis=0)

    train_scores = []
    test_scores = []
    oot_scores = []
    for i in range(1, X_train.columns.shape[0] + 1):
        cols = []
        for k, v in dic.items():
            if k <= i:
                cols.append(v)
        model.fit(X_train[cols], y_train)
        train_score = cross_val_score(model, X_train[cols], y_train, scoring=scoring, cv=5, n_jobs=-1, error_score='raise').mean()  # cross_val_score也是测试集分数
        train_scores.append(train_score)
        test_score = cross_val_score(model, X_test[cols], y_test, scoring=scoring, cv=5, n_jobs=-1, error_score='raise').mean()  # cross_val_score也是测试集分数
        test_scores.append(test_score)
        oot_score = cross_val_score(model, X_oot[cols], y_oot, scoring=scoring, cv=5, n_jobs=-1, error_score='raise').mean()  # cross_val_score也是测试集分数
        oot_scores.append(oot_score)

    scores = pd.DataFrame([])
    scores['train'] = train_scores
    scores['test'] = test_scores
    scores['oot'] = oot_scores
    return feature_ranking, scores

##### 嵌入法  
SelectFromModel没有RFE靠谱  
RFE迭代删除最不重要的特征  
SelectFromModel 的鲁棒性稍差一些，因为它只是根据作为参数给出的阈值删除不太重要的特征。不涉及迭代。

In [ ]:
def feature_selection_embedded(clf, X, y, scoring='roc_auc', threshold=[]):
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.feature_selection import SelectFromModel
    from sklearn.model_selection import cross_val_score, cross_validate

    #! 画学习曲线来找最佳阈值（特征重要性）
    if len(threshold) == 0:
        threshold = np.linspace(
            0, (clf.fit(X, y).feature_importances_).max(), 20)
    scores = []
    for th in threshold:
        X_embedded = SelectFromModel(clf, threshold=th).fit_transform(X, y)
        # score = cross_val_score(xgbc, X_embedded, y, cv=5).mean()
        # scores.append(score)
        score = cross_validate(
            clf, X_embedded, y, scoring=scoring, cv=5, n_jobs=-1, error_score='raise')
        scores.append(score['test_score'].mean())
    plt.plot(threshold, scores)
    plt.show()
    return scores

##### 线性降维

多重共线性分析  
特征变量与其他特征变量之间的相关性系数大，存在较大的共线性影响，会导致模型不准确。可使用降维的方法，去除多重共线性

In [ ]:
def get_vif(train_data):

    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # 方差膨胀因子VIF：如果将i特征添加到线性回归中，则参数估计的方差增加的度量
    # 如果VIF大于5，则i特征与其他特征高度共线，因此参数估计会有很大的标准误差
    X = np.matrix(train_data.values)
    VIF_list = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    return VIF_list

主成分分析

In [ ]:
from sklearn.decomposition import PCA
# 通过某种线性投影，将高维数据映射到低维空间，并期望在所投影的维度上，数据的方差最大，以达到用较少的数据也能保留较多原数据特性的效果
PCA(n_components=0.9)  # 保持90%的信息

线性判别分析

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# PCA是尽可能保留原数据信息，
# LDA是使降维后的数据尽可能的区分开，其利用了标签的信息，使同类的数据尽可能接近，不同类数据尽可能分开
LDA(n_components=2)

### 过程监控

#### 学习曲线  
X轴：训练集样本数，y轴：模型准确率/回归误差，帮助判断方差/偏差是否过高，增大训练集是否减小过拟合

In [ ]:
def plot_learning_curve(estimator, title, X, y, scoring='roc_auc', ax=None, random_state=2022):
    from sklearn.model_selection import learning_curve
    import matplotlib.pyplot as plt
    import numpy as np

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, scoring=scoring, shuffle=True,
        cv=5, random_state=random_state, n_jobs=-1)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    if ax == None:
        ax = plt.gca()
    else:
        ax = plt.figure()
    ax.set_title(title)
    ax.set_ylim(0.0, 1.1)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    ax.grid()  # 绘制网格，不是必须
    ax.fill_between(
        train_sizes, train_scores_mean - train_scores_std, 
        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    ax.fill_between(
        train_sizes, test_scores_mean - test_scores_std, 
        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    ax.plot(train_sizes, np.mean(train_scores, axis=1),
            'o-', color="r", label="Training score")
    ax.plot(train_sizes, np.mean(test_scores, axis=1),
            'o-', color="g", label="Test score")
    ax.legend(loc="best")
    plt.show()
    return ax

#### 验证曲线  
X轴：超参数值，y轴：模型评分，随着超参数变化，模型的变化（比如从欠拟合——合适——过拟合）

In [ ]:
def plot_validation_curve(estimator, title, X, y, param_name='alpha', param_range=[0.1, 0.01, 0.001, 0.0001], scoring='roc_auc'):
    from sklearn.model_selection import validation_curve
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure()
    plt.title(title)
    plt.xlabel(param_name)
    plt.ylabel('Score')
    plt.ylim(0.0, 1.1)
    train_scores, test_scores = validation_curve(
        estimator, X, y, param_name=param_name, param_range=param_range, cv=5, scoring=scoring, n_jobs=-1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.semilogx(param_range, train_scores_mean, label='Train Score', color='r')
    plt.fill_between(
        param_range, train_scores_mean - train_scores_std, 
        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.semilogx(param_range, test_scores_mean, label='Cross-validation Score', color='g')
    plt.fill_between(
        param_range, test_scores_mean - test_scores_std, 
        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    # plt.plot(param_range, train_scores_mean, 'o-', color='r', label='Train Score')
    # plt.plot(param_range, test_scores_mean, 'o-', color='g', label='Cross-validation Score')
    plt.legend(loc='best')
    plt.show()
    return plt

### 模型融合

#### bagging  
1. 先Bootstrap sampling，得到N个包含m个样本的dataset
2. 再训练多个基模型。如随机森林

#### blending
1. 第一层：
- 把Data Set（假设shape=(100, D)）分为两部分，80%作训练集train，20%作测试集test
- 再次把train按5:5分为两部分train1、train2，用train1训练N个模型
- 每一个模型对train2、和test作预测，得到(40, N)和(20, N)
- 再将预测值合并到train2和test，得到(40, D+N)和(20, D+N)
2. 第二层：
- 用得到的新train2(40, D+N)训练模型（或直接用（40, N）训练，（20, N）测试）
- 用(20, D+N)来测试模型预测值与y的差异

优点：
- blending不用做k折交叉验证，获得新的feature  
- 避免一些信息泄漏  

缺点：
- 得到最终模型第二层，样本太少（按7:3划分train和test，再按7:3划分train1和train2，则第二层训练集只有9%的样本）
- 很可能过拟合

#### Stacking

1. 第一步：
- 训练3个基模型，用所有基模型对样本进行预测。第j个基模型对样本i的预测值或矩阵=新的dataset.iloc[i, j]
- 每一个模型：五折交叉验证，训练集前4折训练，第五折验证，model给出第五折验证集的预测值$pred_5$和完整测试集$pred_{test5}$，五折走完得到$pred_1 \dots pred_5$堆叠形成训练集的$train$，对测试集的$pred_{test1} \dots pred_{test5}$按行取平均，得到测试集的$test$
2. 第二步：基于新的训练集训练模型
3个模型训练完后，得到$\begin{pmatrix}  
  train_{1} & train_{2} & train_{3} \\  
  test_{1} & test_{2} & test_{3}
\end{pmatrix} $，使用train训练第二轮模型，使用test测试第二轮模型的效果model2(test)与y的差异

<img src="https://img-blog.csdnimg.cn/20210513224705277.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MjM2NTQ0Mw==,size_16,color_FFFFFF,t_70" style="zoom:50%;"/>

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=2022)

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42)))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=rkf, stack_method='auto', n_jobs=-1, passthrough=False, verbose=0)


In [ ]:
def missrate_by_month(data, month_col, x_cols):
    df = data.groupby(month_col)[x_cols].apply(
    lambda x: x.isna().sum() / len(x))
    df = df.T
    df['miss_rate_d'] = df.std(axis=1)
    return df

from statsmodels.stats.outliers_influence import variance_inflation_factor

[statsmodels.stats.outliers_influence.variance_inflation_factor(df.values,i) for i in range(df.shape[1])]

In [1]:
def variation_by_month(df, time_col, columns, label_col):
    variation_dic = {}
    for col in columns:
        feature_v = df.pivot_table(index=col, columns=time_col, values=label_col, agg_func=['mean'])
        variation_dic[col] = feature_v.rank().std(axis=1).mean()
    return pd.DataFrame([variation_dic], index=['variation']).T  # 设置阈值，如波动值<0.8